In [1]:
!pip install wordcloud

In [2]:
import re, string, unicodedata                                          #Used for Text PreProcessing
import numpy as np 
import random
import pandas as pd 
import matplotlib.pyplot as plt                                         # Used for plotting
import seaborn as sns                                                   # Used for plotting
from collections import Counter                                           # count the key-value pairs in an object

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator           # Used for plotting the wordcloud of a corpus
import nltk                                                               # Used for different task of NLP
from nltk.corpus import stopwords                                         # Used for removal of stop words
import warnings                                                           
warnings.filterwarnings("ignore")
from nltk.stem.porter import PorterStemmer  

In [3]:
data = pd.read_csv('tweets.csv')

In [4]:
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [5]:
data.shape

(14640, 15)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      14640 non-null  int64  
 1   airline_sentiment             14640 non-null  object 
 2   airline_sentiment_confidence  14640 non-null  float64
 3   negativereason                9178 non-null   object 
 4   negativereason_confidence     10522 non-null  float64
 5   airline                       14640 non-null  object 
 6   airline_sentiment_gold        40 non-null     object 
 7   name                          14640 non-null  object 
 8   negativereason_gold           32 non-null     object 
 9   retweet_count                 14640 non-null  int64  
 10  text                          14640 non-null  object 
 11  tweet_coord                   1019 non-null   object 
 12  tweet_created                 14640 non-null  object 
 13  t

In [7]:
#Dropping the unneeded rows and then printing the shape of the new data.
data.drop(['tweet_id','airline_sentiment_confidence','negativereason','negativereason_confidence','airline','airline_sentiment_gold','name','negativereason_gold','retweet_count','tweet_coord','tweet_created','tweet_location','user_timezone'],axis=1,inplace=True)
data.shape

(14640, 2)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   airline_sentiment  14640 non-null  object
 1   text               14640 non-null  object
dtypes: object(2)
memory usage: 228.9+ KB


In [9]:
data.head()

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [10]:
#Import Beautiful Soup Library
import bs4

In [11]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
#Text Preprocessing
def review_to_words(raw_review):
    #Remove HTML tags using the Beautiful Soup Library.
    review_text = BeautifulSoup(raw_review).get_text()  
    #Removing anything that isnt an alphabetical letter.
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #Converting to all lowercase.
    words = letters_only.lower().split()  
    #Searching for stopwords and then removing them.
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops]   
    return(" ".join(meaningful_words))  

In [12]:
nltk.download('stopwords')
num_reviews = data["text"].size
clean_train_reviews = []
for i in range(0,num_reviews):
    clean_train_reviews.append(review_to_words(data["text"][i]))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aryagupta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
#Printing the first 5 rows of the cleaned text reviews.
for i in range(5):
    print(clean_train_reviews[i])

virginamerica dhepburn said
virginamerica plus added commercials experience tacky
virginamerica today must mean need take another trip
virginamerica really aggressive blast obnoxious entertainment guests faces little recourse
virginamerica really big bad thing


In [14]:
#Lemmatization
import spacy
nlp = spacy.load('en_core_web_sm')
def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

In [15]:
lem_text = []
for i in range(0, len(clean_train_reviews)):
    lem_text.append(lemmatize_text(clean_train_reviews[i]))

In [16]:
for i in range(0,5):
    print(lem_text[i])

virginamerica dhepburn say
virginamerica plus add commercial experience tacky
virginamerica today must mean need take another trip
virginamerica really aggressive blast obnoxious entertainment guest face little recourse
virginamerica really big bad thing


In [17]:
#Count Vectorizer
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(lem_text)
vector = vectorizer.transform(lem_text)
print(vector.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [18]:
#Tfidf Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer1 = TfidfVectorizer()
vectorizer1.fit(lem_text)
# summarize
print(vectorizer1.vocabulary_)

{'virginamerica': 11081, 'dhepburn': 2570, 'say': 8955, 'plus': 7839, 'add': 115, 'commercial': 1894, 'experience': 3356, 'tacky': 9966, 'today': 10328, 'must': 6731, 'mean': 6338, 'need': 6838, 'take': 9977, 'another': 437, 'trip': 10471, 'really': 8364, 'aggressive': 183, 'blast': 1048, 'obnoxious': 7195, 'entertainment': 3155, 'guest': 4306, 'face': 3407, 'little': 5926, 'recourse': 8409, 'big': 993, 'bad': 768, 'thing': 10183, 'seriously': 9112, 'would': 11518, 'pay': 7598, 'flight': 3639, 'seat': 9046, 'play': 7802, 'fly': 3689, 'va': 10950, 'yes': 11696, 'nearly': 6835, 'every': 3274, 'time': 10262, 'vx': 11160, 'ear': 2950, 'worm': 11493, 'go': 4136, 'away': 719, 'miss': 6532, 'prime': 8020, 'opportunity': 7336, 'man': 6209, 'without': 11433, 'hat': 4409, 'parody': 7538, 'https': 4720, 'co': 1823, 'mwpg': 6744, 'grezp': 4258, 'well': 11285, 'amazing': 338, 'arrive': 561, 'hour': 4680, 'early': 2951, 'good': 4165, 'know': 5626, 'suicide': 9810, 'second': 9058, 'lead': 5780, 'caus

In [19]:
print(vectorizer1.idf_)

[4.98577902 9.89843391 9.89843391 ... 9.89843391 9.89843391 9.89843391]


In [20]:
vector1 = vectorizer1.transform(lem_text).toarray()

In [21]:
#Fitting model with Count Vectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
forest = RandomForestClassifier(n_estimators = 10,n_jobs=4) 
print ("Training the random forest...")
forest = forest.fit(vector, data['airline_sentiment'])
print (forest)
print (np.mean(cross_val_score(forest,vector,data['airline_sentiment'],cv=10)))

Training the random forest...
RandomForestClassifier(n_estimators=10, n_jobs=4)
0.7063524590163934


In [22]:
#Fitting model with TfIdf Vectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
forest = RandomForestClassifier(n_estimators = 10,n_jobs=4) 
print ("Training the random forest...")
forest = forest.fit(vector1, data['airline_sentiment'])
print (forest)
print (np.mean(cross_val_score(forest,vector1,data['airline_sentiment'],cv=10)))

Training the random forest...
RandomForestClassifier(n_estimators=10, n_jobs=4)
0.6993169398907104
